# HDS5210-2022 Midterm

In the midterm, you're going to use all the programming and data management skills you've developed so far to build a risk calculator that pretends to be integrated with a clinical registry.  You'll compute the PRIEST COVID-19 Clinical Severity Score for a series of patients and, based on their risk of an adverse outcome, query a REST web service to find a hospital to transfer them to. The end result of your work will be a list of instructions on where each patient should be discharged given their risk and various characteristics of the patient.

Each step in the midterm will build up to form your complete solution.

**Make sure you write good docstrings and doctests along the way!!**

**The midterm is due at 11:59 PM CST on Monday, October 23rd.**

---

## Step 1: Calculate PRIEST Clinical Severity Score

This scoring algorithm can be found [here on the MDCalc website](https://www.mdcalc.com/priest-covid-19-clinical-severity-score#evidence).  

1. You will need to write a function called **priest()** with the following input parameters.  
 * Sex (Gender assigned at birth)
 * Age in years
 * Respiratory rate in breaths per minute
 * Oxygen saturation as a percent between 0 and 1
 * Heart rate in beats per minute
 * Systolic BP in mmHg
 * Temperature in degrees C
 * Alertness as a string description
 * Inspired Oxygen as as string description
 * Performance Status as a string description
2. The function will need to follow the algorithm provided on the MDCalc website to compute a risk percentage that should be returned as a numeric value between 0 and 1.
3. Be sure to use docstring documentation and at least three built-in docstring test cases.
4. Assume that the input values that are strings could be any combination of upper or lower case. For example: 'male', 'Male', 'MALE', 'MalE' should all be interpretted by your code as male.
5. If any of the inputs are invalid (for example a sex value that is not recognizable as male or female) your code should return None.

NOTES:
1. In the final step there is a table that translates from **PRIEST Score** to **30-day probability of an outcome** but the last two probabilities are shown as ranges (59-88% and >99%).  Our code needs to output a single number, however. For our code, use the following rule:
 * If PRIEST score is between 17 and 25, the probability you return should be 0.59
 * If PRIEST score is greater than or equal to 26, the probability you return should be 0.99


In [1]:
def priest(sex='male',
       age=40,
       respiratory_rate=18.0,
       oxygen_saturation=0.99,
       heart_rate=65.0,
       systolic_bp=125.0,
       temperature=36.2,
       alertness='ALERT',
       inspired_oxygen='AIR',
       performance_status=	'limited self-care'):
  """

  ((str),(int),(float),(float),(float),(float),(float),(str),(str),(str)) -> float
  here we are are defining to calculate priest score for covid patients. we defined the priest with above parameters. We defaultly kept their values on the above. we dont know how they gonna give their inputs,
  so we are lowering the textcase and getting the inputs in their following variables. Now we are assigning score as zero. Now we start conditioning of every parameter according to the site. if the condition
  satisfied then we increment the score as they said in the site. I also did temperature in two conversions. If the temperature is in centigrade then we do scoring as per that and if temperature is greater than
  90 then we consider it as farenheit scale and do conversion by this formulae (temperature-32)*5/9 then we get the temperature in centigrade and do score as usual. we kept a condition that sex should be only male and
  female and also if spelling was wrong then it has to give output as None. I also kept condition if other than mentioned condition came as input then the result will be None.

  ------Note------
  In the below score for age below 16 is also zero so I am modifying the code in age that age>0 and age<50 the score is zero

  >>> priest('female',32,19,0.95,82,130,96.2,'AlErt','AIr','limited self-care')
  0.09

  >>> priest('Male',56,26,0.92,105,160,39.8,'Supplemental oxygen','bed/chair bound, no self-care')
  0.47

  >>> priest('male',5,16,0.85,100,120,94.2,'Air','limited activity, can self-care')
  0.34

  >>> priest()
  0.03

  this will return normal output because we are using default values in priest score interpretation

  """
  sex=sex.lower()
  alertness=alertness.lower()
  inspired_oxygen=inspired_oxygen.lower()
  performance_status=performance_status.lower()
  if sex!='male' and sex!='female':
    return None

  score=0
  ## SEX ##
  if sex=='male':
    score+=1

  elif sex=='female':
    score+0

  ## AGE ##
  if age is float()==False and age is int()==False:
    return None
  if age>0 and age<50:
    score+=0

  elif age>=50 and age<66:
    score+=2

  elif age>=66 and age<81:
    score+=3

  elif age>80:
    score+=4

  else:
    return None

  ## Respiratory Rate ##
  if respiratory_rate<9:
    score+=3

  elif respiratory_rate>=9 and respiratory_rate<12:
    score+=1

  elif respiratory_rate>=12 and respiratory_rate<21:
    score+=0

  elif respiratory_rate>=21 and respiratory_rate<25:
    score+=2

  elif respiratory_rate>24:
    score+=3

  ## O2 Saturation
  if oxygen_saturation>1:
    oxygen_saturation=oxygen_saturation/100

  if oxygen_saturation >0.95:
    score+=0

  elif oxygen_saturation>=0.94 and oxygen_saturation<=0.95:
    score+=1

  elif oxygen_saturation>=0.92 and oxygen_saturation<=0.93:
    score+=2

  elif oxygen_saturation<0.92:
    score+=3

  ## Heart Rate

  if heart_rate<41:
    score+=3

  elif heart_rate>=41 and heart_rate<=50:
    score+=1

  elif heart_rate>=51 and heart_rate<=90:
    score+=0

  elif heart_rate>=91 and heart_rate<=110:
    score+=1

  elif heart_rate>=111 and heart_rate<=130:
    score+=2

  elif heart_rate>130:
    score+=3

  ## Systolic BP

  if systolic_bp<91:
    score+=3

  elif systolic_bp>=91 and systolic_bp<=100:
    score+=2

  elif systolic_bp>=101 and systolic_bp<=110:
    score+=1

  elif systolic_bp>=111 and systolic_bp<=219:
    score+=0

  elif systolic_bp>219:
    score+=3

  ## Temperature
  if temperature <35.1:
    score+=3

  elif temperature>=35.1 and temperature<=36.0:
    score+=1

  elif temperature>=36.1 and temperature<=38.0:
    score+=0

  elif temperature>=38.1 and temperature<=39.0:
    score+=1

  elif temperature>39.0 and temperature<91 :
    score+=2

  ### if temperature is given in farenheit scale
  elif temperature>90:
    temperature=(temperature-32)*(5/9)

    if temperature <35.1:
      score+=3

    elif temperature>=35.1 and temperature<=36.0:
      score+=1

    elif temperature>=36.1 and temperature<=38.0:
      score+=0

    elif temperature>=38.1 and temperature<=39.0:
      score+=1

    elif temperature>39.0:
      score+=2


  ## Patient Alertness

  if alertness=='alert':
    score+=0

  elif alertness=='confused or not alert':
    score+=3

  ## Inspired Oxygen

  if inspired_oxygen=='air':
    score+=0

  elif inspired_oxygen=='supplemental oxygen':
    score+=2


  ## Performance Status

  if performance_status=='unrestricted normal activity':
    score+=0

  elif performance_status=='limited strenuous activity, can do light activity':
    score+=1

  elif performance_status=='limited activity, can self-care':
    score+=2

  elif performance_status=='limited self-care':
    score+=3

  elif performance_status=='bed/chair bound, no self-care':
    score+=4


    ## Percentage

  if score==0 or score==1:
    percentage=0.01

  elif score==2 or score==3:
    percentage=0.02

  elif score==4:
    percentage=0.03

  elif score==5:
    percentage=0.09

  elif score==6:
    percentage=0.15

  elif score==7:
    percentage=0.18

  elif score==8:
    percentage=0.22

  elif score==9:
    percentage=0.26

  elif score==10:
    percentage=0.29

  elif score==11:
    percentage=0.34

  elif score==12:
    percentage=0.38

  elif score==13:
    percentage=0.46

  elif score==14:
    percentage=0.47

  elif score==15:
    percentage=0.49

  elif score==16:
    percentage=0.55

  elif score>=17 and score<=25:
    percentage=0.59

  elif score>=26:
    percentage=0.99

  return percentage


In [2]:
import doctest
doctest.run_docstring_examples(priest, globals(),verbose=True)


PYDEV DEBUGGER WARNING:
sys.settrace() should not be used when the debugger is being used.
This may cause the debugger to stop working correctly.
If this is needed, please check: 
http://pydev.blogspot.com/2007/06/why-cant-pydev-debugger-work-with.html
to see how to restore the debug tracing back correctly.
Call Location:
  File "/usr/lib/python3.10/doctest.py", line 1501, in run
    sys.settrace(save_trace)



Finding tests in NoName
Trying:
    priest('female',32,19,0.95,82,130,96.2,'AlErt','AIr','limited self-care')
Expecting:
    0.09
ok
Trying:
    priest('Male',56,26,0.92,105,160,39.8,'Supplemental oxygen','bed/chair bound, no self-care')
Expecting:
    0.47
ok
Trying:
    priest('male',5,16,0.85,100,120,94.2,'Air','limited activity, can self-care')
Expecting:
    0.34
ok
Trying:
    priest()     
Expecting:
    0.03
ok


## Part 2: Find a hospital

The next thing we have to do is figure out where to send this particular patient.  The guidelines on where to send a patient are based on their age (pediatric, adult, geriatric), sex, and risk percentage.  Luckily, you don't have to implement these rules. I already have. All you have to do is use a REST web service that I've created for you.

You'll want to use Python to make a call to my REST web service similar to the example URL below. The first part of the URL will be the same for everyone and every request that you make. What you will need to modify for each of your requests is the information after the question mark.

```
https://oumdj6oci2.execute-api.us-east-1.amazonaws.com/prd/?age=40&sex=male&risk_pct=0.1
```

The example above asks my web service where a 40-year old male with a risk of 10% should go.  What the web service will return back is a JSON string containing the information you need.  That JSON will look like this:

```json
{
  "age": "40",
  "sex": "male",
  "risk": "0.1",
  "hospital": "Southwest Hospital and Medical Center"
}
```

My function is not smart enough to understand `'MALE'` is the same as `'male'`.  You have to send it exactly `'male'` or `'female'`

1. Your job is to write a function called **find_hospital()** that takes age, sex, and risk as parameters.
2. Your function should call this REST web service using the `requests` module
3. Then your function will need to interpret the JSON it gets and return just the name of the hospital
4. If anything fails, return None
5. Include a good docstring with at least three test cases.


In [3]:
## Here we are importing requests module and using get to import files from website we were creating the a
import requests
data=requests.get('https://oumdj6oci2.execute-api.us-east-1.amazonaws.com/prd/?age=40&sex=male&risk_pct=0.1')

In [4]:
## This is the the way to import the json file on importing json file we get output as dictionary
import json
data=json.loads(data.text)
data

{'age': '40',
 'sex': 'male',
 'risk': '0.1',
 'hospital': 'Southwest Hospital and Medical Center'}

In [5]:
## Here we defining a function named find_hospital to obtain hospital of patient with following  parameters
def find_hospital(age=55,sex='Male',risk=0.55):
  """

  ((int),(str),(float)) -> str
  Here we defined a function to obtain hospital of the patient using patient age, sex and risk parameters. here we used format function because different url has different informations, the url changes when the
  sex age and risk in url changes. We are going to change the values in the url using this format function. we obtained a url with our input parameter that are already created before. on obtaining the data from
  url using requests module. now using json I loaded the response text.

  I kept conditions to obtain none if sex is other than male or female or if risk >0.99 or risk<0.01 or risk== None.

  >>> find_hospital(age=20,sex='female',risk=0.06)
  'Select Specialty Hospital - Northeast Atlanta'

  >>> find_hospital(age=50,sex='male',risk=0.50)
  'Emory Dunwoody Medical Center'

  >>> find_hospital(age=90,sex='unknown',risk=0.8)==None
  True

  """
  data=requests.get(f'https://oumdj6oci2.execute-api.us-east-1.amazonaws.com/prd/?age={age}&sex={sex}&risk_pct={risk}')
  sex=sex.lower()
  patient_data=json.loads(data.text)
  if sex!='male' and sex!='female':
    return None
  elif risk>0.99 or risk<0.01 or risk==None:
    return None
  else:
    return patient_data['hospital']



In [6]:
find_hospital(age=90,sex='male',risk=0.7)

'Wesley Woods Geriatric Hospital'

In [7]:
import doctest
doctest.run_docstring_examples(find_hospital, globals(),verbose=True)

Finding tests in NoName
Trying:
    find_hospital(age=20,sex='female',risk=0.06)
Expecting:
    'Select Specialty Hospital - Northeast Atlanta'
ok
Trying:
    find_hospital(age=50,sex='male',risk=0.50)
Expecting:
    'Emory Dunwoody Medical Center'
ok
Trying:
    find_hospital(age=90,sex='unknown',risk=0.8)==None
Expecting:
    True
ok


## Part 3: Get the address for that hospital from a JSON file

Great! Now we have code to tell us which hospital to send someone to... but we don't know where that hospital is. The next function we need to create is one that looks up the address of that hospital.  All of these hospitals are in Atlanta, Georgia.  We're going to use the list from this webpage to lookup the address for that hospital, based on its name.  https://www.officialusa.com/stateguides/health/hospitals/georgia.html

Because we skipped the section about Beautiful Soup and working with HTML, I've converted this information into a JSON document for you.  It's available for you here.  Your code should retrieve this file using the `requests` module.

`https://drive.google.com/uc?export=download&id=1fIFD-NkcdiMu941N4GjyMDWxiKsFJBw-`

1. You need to create a function called **get_address()** that takes hospital name as a parameter and searches the data from this JSON file for the hospital you want to find.
2. Your code will have to load the JSON and return the correct hospital based on name.
3. If the hospital name isn't found, the function should return None.
4. Be sure to use good docstring documentation and includes at least 3 test cases.

In [8]:
import requests
import json

def get_address(hospital='abc'):
  """

  (str)-> str
  Here we have a website where there is a json file full of hospital lists with their information. we have initially a dictionary where the keys in it were hospital names with uppercase
  then each hospital values are again a dictionary that contain the hospital details. we need to access that address key values. Initially we are accessing the link using requests. Then we are collecting
  the text from the response. then with open function we are creating hospital.json in write mode as info, then we are writing the text from the response to info where it will be directly saved in hospital.json

  Now we are loading the file by opening using json module. Now the loaded data is a dictionary. we see the input parameter, which is a hospital name, we are going to call hospital name if input hospital name
  is in dictionary keys. If it is present then we call the hospital and access Address key to obtain hospital address.

  >>> get_address('Select Specialty Hospital - Northeast Atlanta')
  '1821 CLIFTON ROAD NE'


  >>> get_address('Emory Dunwoody Medical Center')
  '4500 NORTH SHALLOWFORD ROAD'

  >>> get_address('Wesley Woods Geriatric Hospital')
  '1821 CLIFTON ROAD, NE'

  """


  data=requests.get('https://hds5210-data.s3.amazonaws.com/ga_hospitals.json')

  with open('hospital.json','w') as info:
    info.write(data.text)

  hospitals=json.load(open('hospital.json'))
  hospital_list=hospitals.keys()

  if hospital.upper() in hospital_list:
    return hospitals[hospital.upper()]['ADDRESS']

  else:
    return None

In [9]:
get_address('Wesley Woods Geriatric Hospital')

'1821 CLIFTON ROAD, NE'

In [10]:
import doctest
doctest.run_docstring_examples(get_address, globals(),verbose=True)

Finding tests in NoName
Trying:
    get_address('Select Specialty Hospital - Northeast Atlanta')
Expecting:
    '1821 CLIFTON ROAD NE'
ok
Trying:
    get_address('Emory Dunwoody Medical Center')
Expecting:
    '4500 NORTH SHALLOWFORD ROAD'
ok
Trying:
    get_address('Wesley Woods Geriatric Hospital')
Expecting:
    '1821 CLIFTON ROAD, NE'
ok


## Part 4: Run the risk calculator on a population

At the link below, there is a file called `people.psv`.  It is a pipe-delimited (`|`) file with columns that match the inputs for the PRIEST calculation above.  Your code should use the `requests` module to retrieve the file from this URL.

`https://drive.google.com/uc?export=download&id=1fLxJN9YGUqmqExrilxSS8furwUER5HHh`


In addition, the file has a patient identifier in the first column.

1. Write a function called **process_people()** that takes the file location above as its only parameter. Your Python program should use your code above to process all of these rows, determine the hospital and address, and return a list whose items are a dictionary like this: `{ patient_number: [sex, age, breath, o2sat, heart, systolic, temp, alertness, inspired, status, hospital, address]}`.  Look at the file in Part 5 for what the output looks like.
2. Be sure to use good docstrings, but you don't need any tests in your doc strings.  I've provided those for you withe file in Part 5.


**NOTE** that when running your code for all the 100 records in the `people.psv` file, it may take a few minutes to complete.  You're making multiple calls to the internet for each record, so that can take a little while.


In [11]:
url='https://hds5210-data.s3.amazonaws.com/people.psv'
def process_people(url):
  """

  (str) -> dict
  we are accessing the data in the website above which is a psv file where we have pipeline delimiter. we are using requests module to access the data by loading data from response text.
  there will be many lines in psv file but according to the file there were 100 lines where if we see it in text the lines were seperated using '\n' so we are splitting the data using split function.
  we are using a condition that the line should be seperated at '\n'. Now we are saving them in a variable called lines. we create a empty dictionary for loading the data. Now in each line we have pipeline in
  between each column data. we are going to split the line using split function and split at delimiter '|' after applying for loop for lines. now we can use index to access each and every column related
  patient data. and we also convert the string to integers and floats as following.

  --Note-- we have first row where it is column names and the last row where it is empty row. so we use slicing method to remove the last row and first row [1:-1] but in new link we have only 1st row issue
  so we used [1:]slicing method.

  now in order to obtain hospital name we need to calculate risk using the patient data in their respective rows. we also obtain patient hospital address by using hospital.

  Now we are loading the empty list using key as patient Id and the list of their data as their value.

  Now we are loading that data into a variable called data.


  """
  report=requests.get(url)
  lines=report.text.split('\n')

  lines=lines[1:]
  patients={}
  for line in lines :
    line      = line.split('|')
    sex       = line[1]
    age       = int(line[2])
    breath    = float(line[3])
    o2sat     = float(line[4])
    heart     = float(line[5])
    systolic  = float(line[6])
    temp      = float(line[7])
    alertness = line[8]
    inspired  = line[9]
    status    = line[10]


    score     = priest(sex=sex,
                       age=age,
                       respiratory_rate=breath,
                       oxygen_saturation=o2sat,
                       heart_rate=heart, systolic_bp=systolic,
                       temperature=temp,
                       alertness=alertness,
                       inspired_oxygen=inspired,
                       performance_status=	status)

    hospital  = find_hospital(age=age,sex=sex,risk=score)

    address   = get_address(hospital)

    patients[line[0]]=[sex, age, breath, o2sat, heart, systolic, temp, alertness, inspired, status, score, hospital, address]

  return patients

# Saving the data in data variable
data=process_people(url)

## Part 5: Checking your final results

The final step is to check your results.  You should be able to compare your results to the output in `people_results.json` at the link below.  Write some code to check your results.  This does not need to be a function.

`https://drive.google.com/uc?export=download&id=1gx1SSC20mO5XL6uYD0mdcM_cL91fcIW5`


In [12]:
## We need to call the data from the url above provided. it is a json file.

## In order to load this data we need to first save this file locally. because the above created variable is a dictionary.

## When we load a json file we obtain it as a dictionary.

import requests
import json
url='https://hds5210-data.s3.amazonaws.com/people_results.json'
check=requests.get(url)

## We are creating a new file called people_results.json in write mode as results.

## Now we are writing each and every data in that json file
with open('people_results.json','w') as results:
  results.write(check.text)

# Now locally saved json file is loaded through json module in result.
result=json.load(open('people_results.json'))

## We kept a condition that if both files matches each other then the following string should print. if doesn't matched then the else statement should print.
if result == data:
  print('The Both were Same')

else:
  print('The Both Were Not Same')

The Both were Same


---

## Check your work above

If you didn't get them all correct, take a few minutes to think through those that aren't correct.


## Submitting Your Work

Submit your work as usual into a folder named `midterm`

---